# 3) Model Training & Evaluation

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.graph_objects as go
import optuna
import joblib 
import warnings
warnings.filterwarnings("ignore")

C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset and perform preprocessing (from notebook 2)
df = pd.read_csv('./data/bank_marketing_preprocessed.csv')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,age_group
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,50-59
1,57,services,married,high.school,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,50-59
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,30-39
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,40-49
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0,50-59


In [3]:
# Separate features and target
X = df.drop('y', axis=1)
y = df['y']
# Load the preprocessor pipeline
preprocessor = joblib.load('./models/preprocessor.pkl')
print("Preprocessor pipeline loaded successfully.")

# Split the raw data for a fair validation split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply the preprocessor pipeline to the split data
X_train_transformed = preprocessor.transform(X_train_raw)
X_test_transformed = preprocessor.transform(X_test_raw)


Preprocessor pipeline loaded successfully.


# 1. Hyperparameter Tuning with Optuna

In [4]:
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    model = lgb.LGBMClassifier(**params)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train_transformed, y_train, cv=cv, scoring='roc_auc')
    
    return np.mean(scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\nBest hyperparameters found by Optuna:")
print(study.best_params)

[I 2025-08-14 13:02:24,870] A new study created in memory with name: no-name-5bc2a0f7-ecbf-4757-b64b-e3bf86b4895e
Best trial: 0. Best value: 0.75816:   2%|▏         | 1/50 [00:02<02:23,  2.92s/it]

[I 2025-08-14 13:02:27,792] Trial 0 finished with value: 0.7581598658986768 and parameters: {'n_estimators': 269, 'learning_rate': 0.2560767376596851, 'num_leaves': 79, 'max_depth': 14, 'subsample': 0.6830501029173188, 'colsample_bytree': 0.7792291375098078, 'min_child_samples': 100, 'reg_alpha': 0.8853997738987127, 'reg_lambda': 0.5866419241992956}. Best is trial 0 with value: 0.7581598658986768.


Best trial: 1. Best value: 0.762114:   4%|▍         | 2/50 [00:11<05:06,  6.39s/it]

[I 2025-08-14 13:02:36,604] Trial 1 finished with value: 0.7621137078740393 and parameters: {'n_estimators': 957, 'learning_rate': 0.10091423601487537, 'num_leaves': 70, 'max_depth': 14, 'subsample': 0.6328498870294438, 'colsample_bytree': 0.6336434185356036, 'min_child_samples': 97, 'reg_alpha': 0.7679747801430942, 'reg_lambda': 0.8424781596000473}. Best is trial 1 with value: 0.7621137078740393.


Best trial: 2. Best value: 0.777341:   6%|▌         | 3/50 [00:14<03:47,  4.84s/it]

...

[I 2025-08-14 13:04:51,529] Trial 47 finished with value: 0.7983995011061319 and parameters: {'n_estimators': 317, 'learning_rate': 0.010170725963453538, 'num_leaves': 93, 'max_depth': 6, 'subsample': 0.8345035624582602, 'colsample_bytree': 0.929425584845916, 'min_child_samples': 61, 'reg_alpha': 0.1290662843949476, 'reg_lambda': 0.20584188876931395}. Best is trial 35 with value: 0.7990818631807087.


Best trial: 35. Best value: 0.799082:  98%|█████████▊| 49/50 [02:30<00:03,  3.26s/it]

[I 2025-08-14 13:04:55,193] Trial 48 finished with value: 0.7961765898433049 and parameters: {'n_estimators': 448, 'learning_rate': 0.012353819834427373, 'num_leaves': 94, 'max_depth': 8, 'subsample': 0.8367191430943886, 'colsample_bytree': 0.997748439829562, 'min_child_samples': 63, 'reg_alpha': 0.09958619377208774, 'reg_lambda': 0.179041219825358}. Best is trial 35 with value: 0.7990818631807087.


Best trial: 35. Best value: 0.799082: 100%|██████████| 50/50 [02:31<00:00,  3.04s/it]

[I 2025-08-14 13:04:56,709] Trial 49 finished with value: 0.776540137308745 and parameters: {'n_estimators': 323, 'learning_rate': 0.2605887189041688, 'num_leaves': 74, 'max_depth': 5, 'subsample': 0.8771106620608956, 'colsample_bytree': 0.8829379140925231, 'min_child_samples': 75, 'reg_alpha': 0.4781240035875893, 'reg_lambda': 0.2714909669389315}. Best is trial 35 with value: 0.7990818631807087.

Best hyperparameters found by Optuna:
{'n_estimators': 447, 'learning_rate': 0.01093697962475703, 'num_leaves': 82, 'max_depth': 7, 'subsample': 0.8120523878750876, 'colsample_bytree': 0.878047974397901, 'min_child_samples': 58, 'reg_alpha': 0.0007410392379483786, 'reg_lambda': 0.4512054135333452}


# 2. Train and Evaluate the Final Model

In [12]:
best_params = study.best_params
best_model = lgb.LGBMClassifier(**best_params, objective='binary', random_state=42, n_jobs=-1, verbose=-1)
best_model.fit(X_train_transformed, y_train)

LGBMClassifier(colsample_bytree=0.878047974397901,
               learning_rate=0.01093697962475703, max_depth=7,
               min_child_samples=58, n_estimators=447, n_jobs=-1, num_leaves=82,
               objective='binary', random_state=42,
               reg_alpha=0.0007410392379483786, reg_lambda=0.4512054135333452,
               subsample=0.8120523878750876, verbose=-1)

In [13]:
# Make predictions
y_pred = best_model.predict(X_test_transformed)
y_pred_proba = best_model.predict_proba(X_test_transformed)[:, 1]

# 3. Comprehensive Evaluation

In [14]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      7310
           1       0.68      0.27      0.39       928

    accuracy                           0.90      8238
   macro avg       0.80      0.63      0.67      8238
weighted avg       0.89      0.90      0.88      8238



In [15]:
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
fig = px.imshow(cm, text_auto=True, color_continuous_scale='Blues',
                labels={'x':'Predicted', 'y':'Actual'},
                x=['No', 'Yes'], y=['No', 'Yes'],
                title='Confusion Matrix')
fig.show()


Confusion Matrix:


In [16]:
# ROC Curve and AUC Score
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC curve (area = {roc_auc:.2f})'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Classifier', line=dict(dash='dash')))
fig.update_layout(title='Receiver Operating Characteristic (ROC) Curve',
                  xaxis_title='False Positive Rate', yaxis_title='True Positive Rate',
                  xaxis_range=[0, 1], yaxis_range=[0, 1])
fig.show()

print(f"\nROC AUC Score: {roc_auc:.4f}")


ROC AUC Score: 0.8142


In [17]:
# Save the trained model
joblib.dump(best_model, './models/best_model.pkl')
print("\nTrained model saved as 'best_model.pkl'")


Trained model saved as 'best_model.pkl'


# 4. Feature Importance Analysis

In [19]:
print("\nFeature Importance Analysis:")
feature_importances = best_model.feature_importances_

# Get the feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
importance_df = importance_df.sort_values('importance', ascending=False).reset_index(drop=True)

fig_feat_imp = px.bar(importance_df.head(20), x='importance', y='feature',
                       orientation='h',
                       title='Top 20 Feature Importances',
                       labels={'importance': 'Importance', 'feature': 'Feature Name'})
fig_feat_imp.update_layout(yaxis={'categoryorder':'total ascending'})
fig_feat_imp.show()


Feature Importance Analysis:
